# 99 — Capstone: end-to-end pandas workflow

Goal: generate data → inject realistic messiness → clean/validate → analyze → produce deliverables.


In [ ]:
import pandas as pd

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


import sys
from pathlib import Path

# If running from the repository root, this makes the shared module importable:
shared = Path.cwd() / "docs" / "tutorials" / "python" / "modules" / "pandas" / "shared"
sys.path.insert(0, str(shared))

from make_orders import make_orders

orders = make_orders()
orders.head()

In [ ]:
# Inject messiness
dirty = orders.copy()
dirty.loc[0, "shipping_status"] = " Delivered  "
dirty.loc[1, "shipping_status"] = "In Transit"
dirty.loc[2, "price"] = -10

dirty.loc[3, "customer"] = None

dirty = pd.concat([dirty, dirty.iloc[[10]]], ignore_index=True)  # duplicate row

dirty[["shipping_status","price","customer"]].head(6)

In [ ]:
# Clean + validate
status_map = {"in transit": "in_transit", "in-transit": "in_transit"}

clean = (
    dirty
    .dropna(subset=["customer"]).copy()
    .drop_duplicates(subset=["order_id"], keep="first")
    .assign(
        order_date=lambda d: pd.to_datetime(d["order_date"], errors="coerce"),
        shipping_status=lambda d: d["shipping_status"].astype("string").str.strip().str.lower().replace(status_map),
        price=lambda d: pd.to_numeric(d["price"], errors="coerce"),
        quantity=lambda d: pd.to_numeric(d["quantity"], errors="coerce"),
    )
    .assign(
        is_valid_price=lambda d: d["price"].ge(0) & d["price"].notna(),
        is_valid_qty=lambda d: d["quantity"].ge(0) & d["quantity"].notna(),
    )
)

clean2 = clean.loc[clean["is_valid_price"] & clean["is_valid_qty"]].copy()

assert clean2["order_id"].is_unique

(len(dirty), len(clean2))

In [ ]:
# Deliverable 1: top categories
rev_by_cat = (
    clean2.groupby("category").agg(revenue_total=("revenue", "sum"), orders=("order_id", "nunique"), items=("quantity", "sum"))
    .sort_values("revenue_total", ascending=False)
)
rev_by_cat

In [ ]:
# Deliverable 2: monthly revenue pivot
clean2["order_date"] = pd.to_datetime(clean2["order_date"], errors="coerce")
ts = clean2.dropna(subset=["order_date"]).set_index("order_date").sort_index()

monthly = ts.groupby("category")["revenue"].resample("MS").sum().reset_index()
pivot = monthly.pivot_table(index="order_date", columns="category", values="revenue", aggfunc="sum", fill_value=0)

pivot.tail()